# --- Day 17: Conway Cubes ---

https://adventofcode.com/2020/day/17

In [1]:
path = '../inputs/'

## Part 1

In [2]:
import copy
from pprint import pprint
import numpy as np

In [3]:
def read_initial_state(filename):
    initial_state = [[]]
    
    with open (path + filename) as file:
        for line in file:
            x = [int(c.replace('.', '0').replace('#', '1')) for c in line.strip()]
            initial_state[0].append(x)

    return np.array(initial_state)

read_initial_state('example_initial_state.txt')

array([[[0, 1, 0],
        [0, 0, 1],
        [1, 1, 1]]])

In [4]:
def expand_state_space(state):
    
    len_z = state.shape[0]
    len_y = state.shape[1]
    len_x = state.shape[2]

    new_z = np.zeros(((len_x+2) * (len_y+2))).reshape(1, len_y+2, len_x+2)
    new_y = np.zeros(len_z * (len_x+2)).reshape(len_z, 1, len_x+2)
    new_x = np.zeros(len_z * len_y).reshape((len_z, len_y, 1))

    # Add extra x dimensions
    state = np.append(state, new_x, 2)
    state = np.append(new_x, state, 2)

    # Add extra y dimensions
    state = np.append(state, new_y, 1)
    state = np.append(new_y, state, 1)

    # Add extra z dimensions
    state = np.append(state, new_z, 0)
    state = np.append(new_z, state, 0)

    return state                           

In [5]:
expand_state_space(read_initial_state('example_initial_state.txt'))

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [6]:
def change_state(state):

    new_state = copy.deepcopy(state)
    
    for z in range(1, state.shape[0]-1):
        for y in range(1, state.shape[1]-1):
            for x in range(1, state.shape[2]-1):

                cube_and_neighbors = state[z-1:z+2, y-1:y+2, x-1:x+2]
                num_active = cube_and_neighbors.sum()
                
                # Add 1 to account for the cube in the center being active
                if state[z, y, x] == 1 and not (num_active == 2+1 or num_active == 3+1):
                    new_state[z, y, x] = 0
                elif state[z, y, x] == 0 and num_active == 3:
                    new_state[z, y, x] = 1
    
    return new_state

In [7]:
def run_boot_process(state, cycle_num=0, stop_at=6):
    
    if cycle_num == stop_at:
        return state
    
    else:
        new_state = change_state(expand_state_space(state))
        cycle_num += 1
        return run_boot_process(new_state, cycle_num=cycle_num)        

In [8]:
initial_state = read_initial_state('example_initial_state.txt')
expanded_initial_state = expand_state_space(initial_state)
final_state = run_boot_process(expanded_initial_state) 
final_state.sum() # Should return 112

112.0

In [9]:
initial_state = read_initial_state('initial_state.txt')
expanded_initial_state = expand_state_space(initial_state)
final_state = run_boot_process(expanded_initial_state) 
final_state.sum() 

372.0

## Part 2

In [10]:
def read_initial_state2(filename):
    initial_state = [[]]
    
    with open (path + filename) as file:
        for line in file:
            x = [int(c.replace('.', '0').replace('#', '1')) for c in line.strip()]
            initial_state[0].append(x)

    initial_state = np.array(initial_state)
    initial_state = initial_state[None, ...] # Add w dimension
        
    return initial_state
    
read_initial_state2('example_initial_state.txt')

array([[[[0, 1, 0],
         [0, 0, 1],
         [1, 1, 1]]]])

In [11]:
def expand_state_space2(state):
    
    len_w = state.shape[0]
    len_z = state.shape[1]
    len_y = state.shape[2]
    len_x = state.shape[3]

    new_w = np.zeros((len_z+2) * (len_y+2) * (len_x+2)).reshape(1, len_z+2, len_y+2, len_x+2)
    new_z = np.zeros(len_w * (len_y+2) * (len_x+2)).reshape(len_w, 1, len_y+2, len_x+2)
    new_y = np.zeros(len_w * len_z * (len_x+2)).reshape(len_w, len_z, 1, len_x+2)
    new_x = np.zeros(len_w * len_z * len_y).reshape(len_w, len_z, len_y, 1)

    # Add extra x dimensions
    state = np.append(state, new_x, 3)
    state = np.append(new_x, state, 3)
    
    # Add extra y dimensions
    state = np.append(state, new_y, 2)
    state = np.append(new_y, state, 2)

    # Add extra z dimensions
    state = np.append(state, new_z, 1)
    state = np.append(new_z, state, 1)

    # Add extra w dimensions
    state = np.append(state, new_w, 0)
    state = np.append(new_w, state, 0)
    
    return state                           

In [12]:
def change_state2(state):

    new_state = copy.deepcopy(state)
    for w in range(1, state.shape[0] - 1):
        for z in range(1, state.shape[1] - 1):
            for y in range(1, state.shape[2] - 1):
                for x in range(1, state.shape[3] - 1):

                    cube_and_neighbors = state[w-1:w+2, z-1:z+2, y-1:y+2, x-1:x+2]
                    num_active = cube_and_neighbors.sum()

                    # Add 1 to account for the cube in the center being active
                    if state[w, z, y, x] == 1 and not (num_active == 2+1 or num_active == 3+1):
                        new_state[w, z, y, x] = 0
                    elif state[w, z, y, x] == 0 and num_active == 3:
                        new_state[w, z, y, x] = 1
    
    return new_state

In [13]:
def run_boot_process2(state, cycle_num=0, stop_at=6):
    
    if cycle_num == stop_at:
        return state
    
    else:
        new_state = change_state2(expand_state_space2(state))
        cycle_num += 1
        return run_boot_process2(new_state, cycle_num=cycle_num)  

In [14]:
initial_state2 = read_initial_state2('example_initial_state.txt')
expanded_initial_state2 = expand_state_space2(initial_state2)
final_state2 = run_boot_process2(expanded_initial_state2) 
final_state2.sum() # Should return 848

848.0

In [15]:
initial_state2 = read_initial_state2('initial_state.txt')
expanded_initial_state2 = expand_state_space2(initial_state2)
final_state2 = run_boot_process2(expanded_initial_state2) 
final_state2.sum() 

1896.0